# Retrieve & Re-Rank Demo over Simple Wikipedia

This examples demonstrates the Retrieve & Re-Rank Setup and allows to search over [Simple Wikipedia](https://simple.wikipedia.org/wiki/Main_Page).

You can input a query or a question. The script then uses semantic search
to find relevant passages in Simple English Wikipedia (as it is smaller and fits better in RAM).

For semantic search, we use `SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')` and retrieve
32 potentially passages that answer the input query.

Next, we use a more powerful CrossEncoder (`cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')`) that
scores the query and all retrieved passages for their relevancy. The cross-encoder further boost the performance,
especially when you search over a corpus for which the bi-encoder was not trained for.


In [ ]:
%%capture
!pip install -U sentence-transformers rank_bm25

In [ ]:
%%capture
!pip install datasets

##Using Bi-Encoder for semantic search on Wikipedia movie plots."

In [ ]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch
from datasets import load_dataset

if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")

ds = load_dataset("Coder-Dragon/wikipedia-movies", split='train[:1000]')

#We use the Bi-Encoder to encode all passages, so that we can use it with semantic search
bi_encoder = SentenceTransformer('nq-distilbert-base-v1')
bi_encoder.max_seq_length = 256     #Truncate long passages to 256 tokens
top_k = 32                          #Number of passages we want to retrieve with the bi-encoder

#The bi-encoder will retrieve 100 documents. We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

# As dataset, we use Simple English Wikipedia. Compared to the full English wikipedia, it has only
# about 170k articles. We split these articles into paragraphs and encode them with the bi-encoder


passages = []
for movie in ds:
   passages.append([movie['Title'], movie['Plot']])


print("Passages:", len(passages))

# We encode all passages into our vector space. This takes about 5 minutes (depends on your GPU speed)
corpus_embeddings = bi_encoder.encode(passages, convert_to_tensor=True, show_progress_bar=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Passages: 1000


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
ds[0]

{'Release Year': 1901,
 'Title': 'Kansas Saloon Smashers',
 'Origin/Ethnicity': 'American',
 'Director': 'Unknown',
 'Cast': None,
 'Genre': 'unknown',
 'Wiki Page': 'https://en.wikipedia.org/wiki/Kansas_Saloon_Smashers',
 'Plot': "A bartender is working at a saloon, serving drinks to customers. After he fills a stereotypically Irish man's bucket with beer, Carrie Nation and her followers burst inside. They assault the Irish man, pulling his hat over his eyes and then dumping the beer over his head. The group then begin wrecking the bar, smashing the fixtures, mirrors, and breaking the cash register. The bartender then sprays seltzer water in Nation's face before a group of policemen appear and order everybody to leave.[1]",
 'Image': 'upload.wikimedia.org/wikipedia/commons/2/2d/KansasSaloonSmashers1901.jpg'}

##BM25 for lexical search: lower case, stop-words removed.

In [ ]:
# We also compare the results to lexical search (keyword search). Here, we use
# the BM25 algorithm which is implemented in the rank_bm25 package.

from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string
from tqdm.autonotebook import tqdm
import numpy as np


# We lower case our text and remove stop-words from indexing
def bm25_tokenizer(text):
    tokenized_doc = []
    for token in text.lower().split():
        token = token.strip(string.punctuation)

        if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
            tokenized_doc.append(token)
    return tokenized_doc


tokenized_corpus = []
for passage in tqdm(passages):
    tokenized_corpus.append(bm25_tokenizer(passage[1]))

bm25 = BM25Okapi(tokenized_corpus)


  0%|          | 0/1000 [00:00<?, ?it/s]

##Wikipedia passage search with lexical and semantic methods ranked.

In [ ]:
# This function will search all wikipedia articles for passages that
# answer the query
def search(query):
    print("Input question:", query)

    ##### BM25 search (lexical search) #####
    bm25_scores = bm25.get_scores(bm25_tokenizer(query))
    top_n = np.argpartition(bm25_scores, -5)[-5:]
    bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)

    print("Top-5 lexical search (BM25) hits")
    for hit in bm25_hits[0:5]:
        print("\t{:.3f}\t{}\n\t\t{}".format(hit['score'], passages[hit['corpus_id']][0], passages[hit['corpus_id']][1].replace("\n", " ")))

    ##### Semantic Search #####
    # Encode the query using the bi-encoder and find potentially relevant passages
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    question_embedding = question_embedding.cuda()
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query

    ##### Re-Ranking #####
    # Now, score all retrieved passages with the cross_encoder
    cross_inp = [[query, passages[hit['corpus_id']][1]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)

    # Sort results by the cross-encoder scores
    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]

    # Output of top-5 hits from bi-encoder
    print("\n-------------------------\n")
    print("Top-5 Bi-Encoder Retrieval hits")
    hits = sorted(hits, key=lambda x: x['score'], reverse=True)
    for hit in hits[0:5]:
         print("\t{:.3f}\t{}\n\t\t{}".format(hit['score'], passages[hit['corpus_id']][0], passages[hit['corpus_id']][1].replace("\n", " ")))

    # Output of top-5 hits from re-ranker
    print("\n-------------------------\n")
    print("Top-5 Cross-Encoder Re-ranker hits")
    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    for hit in hits[0:5]:
        print("\t{:.3f}\t{}\n\t\t{}".format(hit['cross-score'], passages[hit['corpus_id']][0], passages[hit['corpus_id']][1].replace("\n", " ")))


In [ ]:
search(query = "Documentaries showcasing indigenous peoples' survival and daily life in Arctic regions")

Input question: Documentaries showcasing indigenous peoples' survival and daily life in Arctic regions
Top-5 lexical search (BM25) hits
	9.210	I Do
		The Boy meets and marries The Girl. A year later, the two walk down the street with a baby carriage carrying a bottle instead of a baby when they run into The Girl's brother who asks the couple to do him a favor and babysit his children. They accept and the remainder of the short consists of gags showcasing the difficulties of babysitting children. At the very end, The Boy discovers some knitted baby clothes in a drawer (implying that The Girl is pregnant).
	8.273	Uncharted Seas
		As described in a film magazine,[3] after her drunken husband Tom Eastman (Gerard) brings home three cabaret women, Lucretia (Lake) can no longer bear the abuse and turns to arctic explorer Frank Underwood (Valentino), who has long loved her and promised to come whenever she needs his help. Urging her husband to become a man and do something worth wile, Lucretia

In [ ]:
search(query = "Western romance")

Input question: Western romance
Top-5 lexical search (BM25) hits
	6.379	The Call of the Wild
		A white girl (Florence Lawrence) rejects a proposal from an Indian brave (Charles Inslee) in this early one-reel Western melodrama. Despite the rejection, the Indian still comes to the girl's defense when she is abducted by his warring tribe. In her first year in films, Florence Lawrence was already the most popular among the Biograph Company's anonymous stock company players. By 1909, she was known the world over as "The Biograph Girl."
	5.688	Romance
		As described in a film publication,[2] a youth (Arthur Rankin) in the prologue seeks advice from his grandfather (Sydney), who then recalls a romance of his own youth which is then shown as a flashback. A priest (Sydney) is in love with an Italian opera singer (Keane), and the drama involves the conflict between his efforts to rise above worldly things or to leave with her. The romance ends with a deep note of pathos.
	5.641	Four Sons
 Franz 

In [ ]:
search(query = "Silent film about a Parisian star moving to Egypt, leaving her husband for a baron, and later reconciling after finding her family in poverty in Cairo.")

Input question: Silent film about a Parisian star moving to Egypt, leaving her husband for a baron, and later reconciling after finding her family in poverty in Cairo.
Top-5 lexical search (BM25) hits
	35.163	Sahara
		Silent film femme fatale, Louise Glaum, portrays the role of Mignon, a Parisian music hall celebrity. Mignon marries a young American civil engineer, John Stanley, portrayed by Matt Moore. Stanley is transferred to Egypt to work on an engineering project in the Sahara. Mignon and her son, portrayed by Pat Moore, join Stanley in the desert.[3][4] Unhappy with life in the desert, Mignon leaves Stanley and her son in the desert and moves to Cairo with the wealthy Baron Alexis, portrayed by Edwin Stevens. Mignon lives in Baron Alexis' palace while Stanley goes blind and becomes addicted to the drug hasheesh. Mignon later encounters Stanley and her son, who have become beggars in the streets of Cairo.[3][4] Mignon returns to the desert to care for her husband, and the two are 

In [ ]:
search(query = "Comedy film, office disguises, boss's daughter, elopement.")

Input question: Comedy film, office disguises, boss's daughter, elopement.
Top-5 lexical search (BM25) hits
	13.201	Mabel's Blunder
		Mabel's Blunder tells the tale of a young woman who is secretly engaged to the boss's son.[1] The young man's sister comes to visit at their office, and a jealous Mabel, not knowing who the visiting woman is, dresses up as a (male) chauffeur to spy on them.
	9.653	Bucking Broadway
		As described in a film magazine,[3] Cheyenne Harry (Carey), one of the cowboys on a ranch in Wyoming, falls in love with Helen (Malone), his boss's daughter. She decides to elope to the city with Captain Thornton (Pegg), a wealthy visitor to the ranch from New York. Cheyenne and Helen's father (Wells) are downhearted. Cheyenne, devastated by the loss of his fiance, decides to go to the city to rescue her, and finds Thorton giving a dinner party in a hotel about to announce his engagement to Helen. As the dinner progresses Helen discovers the true nature of Thornton and endeav

In [ ]:
search(query = "Lost film, Cleopatra charms Caesar, plots world rule, treasures from mummy, revels with Antony, tragic end with serpent in Alexandria.")

Input question: Lost film, Cleopatra charms Caesar, plots world rule, treasures from mummy, revels with Antony, tragic end with serpent in Alexandria.
Top-5 lexical search (BM25) hits
	70.717	Cleopatra
 Cleopatra (Bara), the Siren of Egypt, by a clever ruse reaches Caesar (Leiber) and he falls victim to her charms. They plan to rule the world together, but then Caesar falls. Cleopatra's life is desired by the church, as the wanton woman's rule has become intolerable. Pharon (Roscoe), a high priest, is given a sacred dagger to take her life. He gives her his love instead and, when she is in need of some money, leads her to the tomb of his ancestors, where she tears the treasure from the breast of the mummy. With this wealth she goes to Rome to meet Antony (Hall). He leaves the affairs of state and travels to Alexandria with her, where they revel. Antony is recalled to Rome and married to Octavia (Blinn), but his soul cries out for Cleopatra. He sends her a message to arm her ships and m

In [ ]:
search(query = "Denis Gage Deane-Tanner")

Input question: Denis Gage Deane-Tanner
Top-5 lexical search (BM25) hits
	29.270	Captain Alvarez
		A melodrama about an American who becomes a revolutionary leader battling evil government spies in Argentina. William Desmond Taylor portrays the title role, and Denis Gage Deane-Tanner, Taylor's younger brother, is thought to have played the small role of a blacksmith.
	4.105	Hangman's House
 Hogan escapes from prison and a gunfight erupts between his men and the guards. Later Dermot and Connaught visit Hogan's hideout and Hogan reveals that his sister died following D'Arcy's desertion. Connaught returns to Hangman's House to discover that D'Arcy has returned. After a struggle she flees to Dermot's house. Hogan and Dermot go to Hangman's House and confront D'Arcy. During a fight between the men a fire breaks out and burns down the house. Hogan and Dermot escape but D'Arcy falls to his death as a balcony collapses. Connaught and Dermot see Hogan off at the port as he returns to Algiers. C

##BM25 Model Evaluation

In [ ]:
# Recall@1

(0 + 0 + 1 + 0 + 1 + 1)/6

0.5

In [ ]:
# MRR

(0 + 0 + 1 + (1/3) + 1 + 1)/6

0.5555555555555555

##ReRanker Model Evaluation

In [ ]:
# Recall@1

(1 + 0 + 1 + 1 + 1 + 0)/6

0.6666666666666666

In [ ]:
# MRR

(1 + 0 + 1 + 1 + 1 + 0)/6

0.6666666666666666